In [110]:
import pandas as pd
from pathlib import Path
import re

docs_path = Path('/home/dev/masters-thesis/documents')
qna_path = docs_path / 'QnA_OSM.xlsx'
latex_path = Path('/home/dev/masters-thesis/tex/generated')

questions_df = pd.read_excel(qna_path, sheet_name='Questions').dropna(how='all')
tests_df = pd.read_excel(qna_path, sheet_name='Tests').dropna(how='all')
levels_df = pd.read_excel(qna_path, sheet_name='Prompt Levels').dropna(how='all')
levels_tests_df = pd.read_excel(
    qna_path, sheet_name='PL Tests').dropna(how='all')

In [111]:
def post_processing_latex(latex_code: str):
    latex_code = latex_code.replace('_', '\\_').replace('$', '\\$')
    
    parts = re.split(
        r'(\\endfirsthead|\\endhead|\\endfoot|\\endlastfoot)', latex_code)

    processed_parts = []
    for part in parts:
        if "\\toprule" in part:
            split_middle = '\n\midrule\n'
            split_top = '\\toprule\n'
            top, mid = part.split(split_middle)
            start, columns = top.split(split_top)
            columns = re.sub('(\w)+', '\\\\textbf{\g<0>}', columns)
            part = split_middle.join([split_top.join([start, columns]), mid])

        processed_parts.append(part)

    result = ''.join(processed_parts)
    return result


def save_latex(filename: str, content: str): 
    with open(latex_path / filename, 'w') as file: 
        file.write(content)

## Questions

In [115]:
latex_str = questions_df.to_latex(
    index=False,
    columns=['Query ID', 'Query', 'Correct Response'],
    longtable=True,
    column_format='p{4cm}p{4.5cm}p{4.5cm}',
    label='tbl:questions-quantitative',
    caption='Questions for quantitative tests'
)

post_processed_latex = post_processing_latex(latex_str)
save_latex('questions.tex', post_processed_latex)

## Test Results

In [113]:
latex_str = tests_df.to_latex(
    index=False,
    columns=['Query ID', 'Agent Type', 'Outcome', 'Duration [s]', 'Tokens'],
    longtable=True,
    column_format='lp{1.8cm}p{1.8cm}p{1.8cm}p{1.8cm}p{1.8cm}',
    float_format='{:.2f}'.format,
    caption='Test results for quantitative tests',
    label='tbl:test-results-quantitative'
)

post_processed_latex = post_processing_latex(latex_str)
save_latex('quantatitive_tests.tex', post_processed_latex)